In [12]:
import pandas as pd
import numpy as np
from datetime import timedelta

# https://dash.plotly.com/installation
# https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e
from sqlalchemy import create_engine
import mysql.connector #  pip3 install mysql-connector-python
import configparser as cp

import plotly.express as px
from jupyter_dash import JupyterDash
# import dash_core_components as dcc # deprecated
#import dash_html_components as html
from dash import html,dash_table, dcc
from dash.dependencies import Input,Output

In [2]:
host_name='ec2-3-12-198-175.us-east-2.compute.amazonaws.com'

## Functions 

In [3]:
def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

# Data Loading

## 01 DB connection 

In [4]:
config=cp.ConfigParser()
config.read('/home/ubuntu/cert/db_login.txt')
db_config=config['ivan_db']


## 2. db connection
engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                             db_config['pwd'],
                                                                             db_config['hostname'],
                                                                             'STOCK_PRED'
                                                                            ))

## 02 Overall Stock 

### 02.1 last 1 year 

In [33]:
## Data loading
df=pd.read_sql("""SELECT *
                  FROM STOCK_PRED.ALL_STOCK_HIST
                  WHERE DATE>=CURDATE()-INTERVAL 360 DAY""",
              con=engine)
df.loc[:,'Close_rf']=[round(x,0) for x in df.Close]

print(df.shape)
print(df.Stock.nunique())
print(df.REFRESH_DATE.max())
df.tail(5)

(1414129, 10)
5991
2022-06-16 02:53:30


,Date,Open,High,Low,Close,Volume,SE,Stock,REFRESH_DATE,Close_rf
1414124,2022-06-15,7.900000,8.050000,7.900,7.970000,89000.0,NYSE,ZTR,2022-06-16 02:53:30,8.0
1414125,2022-06-15,159.139999,160.229996,155.500,158.070007,1758000.0,NYSE,ZTS,2022-06-16 02:53:30,158.0
1414126,2022-06-15,8.760000,9.075000,8.586,8.910000,1460800.0,NYSE,ZUO,2022-06-16 02:53:30,9.0
1414127,2022-06-15,2.620000,2.865000,2.550,2.830000,182000.0,NYSE,ZVIA,2022-06-16 02:53:30,3.0
1414128,2022-06-15,5.840000,5.910000,5.490,5.800000,920600.0,NYSE,ZYME,2022-06-16 02:53:30,6.0


### 02.2 Last 40 days 

In [34]:
df_l40=df.loc[df.Date>=df.Date.max()-timedelta(days=60),:]
print(df_l40.shape)
print(df_l40.Stock.nunique())
print(df_l40.Date.min(),
      df_l40.Date.max()
     )

(223415, 10)
5633
2022-04-18 00:00:00 2022-06-15 00:00:00


## 03 Linear regression summary table 

In [7]:
df_lr=pd.read_sql("""SELECT *
                     FROM STOCK_PRED.LINEAR_REG_L40
                     WHERE MODEL_DATE IN (SELECT MAX(MODEL_DATE) FROM STOCK_PRED.LINEAR_REG_L40)
                     ORDER BY MODEL_DATE DESC, WT_COEF DESC""",
                  con=engine
                 )
print(df_lr.shape)
print(df_lr.Stock.nunique())
df_lr.groupby('Model_date').size()

(5550, 14)
5550


Model_date
2022-06-15 19:54:50    5550
dtype: int64

In [8]:
df_lr.head(5)

,Model_date,Stock,Name,Industry,R_squared,Coef,P_values,WT_Coef,Start_Date,End_Date,start_price,end_price,Num_records_dist,growth_rate
0,2022-06-15 19:54:50,WTM,White Mountains Insurance Group Ltd. Common Stock,Property-Casualty Insurers,0.830563,6.395319,3.151133e-16,5.311716,2022-04-18,2022-06-15,1089.469971,1241.699951,40,0.139728
1,2022-06-15 19:54:50,TPL,Texas Pacific Land Corporation Common Stock,Oil & Gas Production,0.497106,8.513429,3.789842e-07,4.232079,2022-04-18,2022-06-15,1488.810059,1624.000000,40,0.090804
2,2022-06-15 19:54:50,COKE,Coca-Cola Consolidated Inc. Common Stock,Beverages (Production/Distribution),0.693180,3.926556,2.721837e-11,2.721812,2022-04-18,2022-06-15,475.089996,551.000000,40,0.159780
3,2022-06-15 19:54:50,BHVN,Biohaven Pharmaceutical Holding Company Ltd. C...,Major Pharmaceuticals,0.655971,1.571029,2.457676e-10,1.030550,2022-04-18,2022-06-15,104.510002,144.979996,40,0.387236
4,2022-06-15 19:54:50,AZPN,Aspen Technology Inc. Common Stock,EDP Services,0.731673,1.367804,4.064692e-12,1.000786,2022-04-18,2022-06-15,161.039993,185.229996,39,0.150211


In [9]:
df_lr_filtered=df_lr.loc[(df_lr.R_squared>0.85)&(df_lr.Coef>0),
                         ['Model_date','Stock','Name','Industry','R_squared','Coef','P_values',
                          'start_price','end_price','Num_records_dist','growth_rate'
                         ]].reset_index(drop=True)
print(df_lr_filtered.shape)
print(df_lr_filtered.Stock.nunique())
df_lr_filtered.head(5)

(3, 11)
3


,Model_date,Stock,Name,Industry,R_squared,Coef,P_values,start_price,end_price,Num_records_dist,growth_rate
0,2022-06-15 19:54:50,STNG,Scorpio Tankers Inc. Common Shares,Marine Transportation,0.933974,0.438598,4.987299e-24,22.709999,36.57,40,0.610304
1,2022-06-15 19:54:50,ULH,Universal Logistics Holdings Inc. Common Stock,Trucking Freight/Courier Services,0.921055,0.324302,1.497310e-22,18.580000,27.74,40,0.493003
2,2022-06-15 19:54:50,DGICA,Donegal Group Inc. Class A Common Stock,Property-Casualty Insurers,0.931653,0.092455,9.625712e-24,13.240000,16.24,40,0.226586


In [25]:
px.scatter(df_lr,
           x='Coef',y='R_squared',color='growth_rate',
           title='Sotck Linear Regression Analysis'
                     )

In [24]:
px.scatter(df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),:],
           x='R_squared',y='Coef',color='growth_rate',size='end_price',
           hover_data=['Stock']  
          )

# Dashboard
- https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e

## import packages 

In [11]:
import plotly.express as px
from jupyter_dash import JupyterDash
# import dash_core_components as dcc # deprecated
#import dash_html_components as html
from dash import html,dash_table, dcc
from dash.dependencies import Input,Output

In [ ]:
host_name='ec2-3-12-111-80.us-east-2.compute.amazonaws.com'

## Dashboard body - 1

In [17]:
# buid app
app=JupyterDash(__name__)

fig=px.scatter(df_lr,
           x='R_squared',y='Coef',color='growth_rate',
           title='Sotck Linear Regression Analysis'
                     )

app.layout=html.Div([html.H1('Stock Linear Regression Analysis'),
                     dcc.Graph(id='graph',
                               figure=fig
                              )])





## run app
app.run_server(mode='external',host='ec2-52-14-195-139.us-east-2.compute.amazonaws.com',debug=True)

KeyboardInterrupt: 

## Dashboard body -2 

In [20]:
app=JupyterDash(__name__)

ln_reg_plot=px.scatter(df_lr,
                       x='R_squared',y='Coef'#,size='growth_rate',color='Industry'#
                       ,hover_name='Name'
                      )

app.layout=html.Div([
    html.Div([
        html.H1('Summary Table'),
        dash_table.DataTable(df_lr.head(10).to_dict('records'),
                             [{'name':i, 'id':i} for i in df_lr.columns]
                            )
    ],style={'display':'inline-block'}),
    html.Div([
        html.H1('Coef VS R_squared Distribution'),
        dcc.Graph(figure=ln_reg_plot,style={'display':'inline-block'})
        
    ]),
    html.Div([
        html.Label('Pick Stock'),
        dcc.Dropdown(['CRM','MSFT','APPL'],'CRM')
    ])
    
])

#html.Div(children=[
#        dcc.Graph(id="graph1", style={'display': 'inline-block'}),
#        dcc.Graph(id="graph2", style={'display': 'inline-block'})
#    ])

if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True)
    
    

Dash app running on http://ec2-3-12-111-80.us-east-2.compute.amazonaws.com:8050/


## Dashboard body -3 

In [48]:
app=JupyterDash(__name__)


app.layout=html.Div([
    ## left ##
    html.Div([
        dcc.Dropdown(
            options=df_l40.Stock.unique(),
            value='CRM',
            id='pick-stock'
        ),
        html.Div([
            html.H1('Best Performing Stocks'),
            dash_table.DataTable(df_lr_filtered.to_dict('records'),
                                 [{'name':i,'id':i} for i in df_lr_filtered.columns]
                                )
        ],
        style={'displace':'inline-block'}
        ),
        html.Div([
            html.H1('Coef vs R_squared'),
            dcc.Graph(id='coef_r_scatter' ,
                      figure=px.scatter(df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),:],
                                        x='R_squared',y='Coef',
                                        color='growth_rate',size='end_price',
                                        hover_data=['Stock']  
                                       )
                      )
        ],
        style={'displace':'inline-block'}
        )
        
    ]),
    
    ## right ##
    html.Div([
        # line chart: last 40 days
        html.Div([
            html.H1('Stock Price Last 40 Days'),
            dcc.Graph(id='stock_price_line_l40')
        ]),
        # line chart: last year
        html.Div([
            html.H1('Stock Price Last Year'),
            dcc.Graph(id='stock_price_line')
        ])
        
    ])
])


@app.callback(
    Output('stock_price_line_l40','figure'),
    Input('pick-stock','value')
)
def update_l40_trend(pick_stock):
    fig_l40=px.line(df_l40.loc[df_l40.Stock==pick_stock,:],
                    x='Date',y='Close',
                    text='Close_rf',
                    title='{} Last 40 Days Stock Price'.format(pick_stock)
                   
                   )
    return fig_l40


@app.callback(
    Output('stock_price_line','figure'),
    Input('pick-stock','value')
)
def update_la_trend(pick_stock):
    fig_la=px.line(df.loc[df.Stock==pick_stock,:],
                   x='Date',y='Close',
                   title='{} Last 1 Year Stock Price'.format(pick_stock)
                  )
    return fig_la

if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True
                  )


/home/ubuntu/anaconda3/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Dash app running on http://ec2-3-12-198-175.us-east-2.compute.amazonaws.com:8050/


In [26]:
if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True)

,Date,Open,High,Low,Close,Volume,SE,Stock,REFRESH_DATE
1190714,2022-04-18,1.220000,1.238500,1.180000,1.190000,18812.0,NASDAQ,AACG,2022-04-19 02:56:26
1190715,2022-04-18,18.860001,19.089100,18.469999,18.540001,27478852.0,NASDAQ,AAL,2022-04-19 02:56:26
1190716,2022-04-18,3.370000,3.370000,3.111200,3.170000,9325.0,NASDAQ,AAME,2022-04-19 02:56:26
1190717,2022-04-18,3.050000,3.110000,2.950000,3.040000,291944.0,NASDAQ,AAOI,2022-04-19 02:56:26
1190718,2022-04-18,51.700001,52.560001,51.525002,51.919998,139185.0,NASDAQ,AAON,2022-04-19 02:56:26


## Dashboard body - 3.1 

In [50]:
app=JupyterDash(__name__)


app.layout=html.Div([
    ## left ##
    html.Div([
        dcc.Dropdown(
            options=df_l40.Stock.unique(),
            value='CRM',
            id='pick-stock'
        ),
        html.Div([
            dash_table.DataTable(df_lr_filtered.to_dict('records'),
                                 [{'name':i,'id':i} for i in df_lr_filtered.columns]
                                ),
            dcc.Graph(id='stock_price_line_l40')
            
        ],style={'displace':'inline-block'}
        )
        
    ]),
    
    ## right ##
    html.Div([
        # line chart: last 40 days
        dcc.Graph(id='coef_r_scatter' ,
                      figure=px.scatter(df_lr.loc[(df_lr.Coef>-5)&(df_lr.growth_rate<2),:],
                                        x='R_squared',y='Coef',
                                        color='growth_rate',size='end_price',
                                        hover_data=['Stock']  
                                       )
                      ),
        
        dcc.Graph(id='stock_price_line')    
        ],style={'displace':'inline-block','float':'right'})
])


@app.callback(
    Output('stock_price_line_l40','figure'),
    Input('pick-stock','value')
)
def update_l40_trend(pick_stock):
    fig_l40=px.line(df_l40.loc[df_l40.Stock==pick_stock,:],
                    x='Date',y='Close',
                    text='Close_rf',
                    title='{} Last 40 Days Stock Price'.format(pick_stock)
                   
                   )
    return fig_l40


@app.callback(
    Output('stock_price_line','figure'),
    Input('pick-stock','value')
)
def update_la_trend(pick_stock):
    fig_la=px.line(df.loc[df.Stock==pick_stock,:],
                   x='Date',y='Close',
                   title='{} Last 1 Year Stock Price'.format(pick_stock)
                  )
    return fig_la

if __name__=='__main__':
    app.run_server(mode='external',
                   host=host_name,
                   debug=True
                  )

Dash app running on http://ec2-3-12-198-175.us-east-2.compute.amazonaws.com:8050/


/home/ubuntu/anaconda3/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

